In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from textblob import TextBlob
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from nltk.tokenize import WhitespaceTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams

In [2]:
df_yelp = pd.read_csv("yelp_new.csv").reset_index(drop = True)

In [4]:
df_yelp.head()

,review_id,user_id,business_id,review_rating,date,text,useful,funny,cool,name,...,longitude,stars,review_count,is_open,categories,text_nopunct,text_lower,text_nostopwords,text_lemmatized,Positivity
0,rv-GQvyaeCASy2vYEJjGOA,qVRavGPFSpCarN21vCOHRw,X2frauzhGG3vOmXpcb5cFQ,5.0,2014-08-04,"I've been to a few movie festivals in Europe, ...",0.0,0.0,1.0,"""Festival Fantasia""",...,-73.575851,5.0,7.0,1.0,Arts & Entertainment;Cinema,Ive been to a few movie festivals in Europe an...,"['ive', 'been', 'to', 'a', 'few', 'movie', 'fe...","['ive', 'movie', 'festivals', 'europe', 'first...","[""['ive',"", ""'movie',"", ""'festivals',"", ""'euro...",1
1,5h6IB5PRrENw4nJD0trfjw,TOunAZFMiaKHIoxCqt5XKA,A4zLP5AyKEEHQr_dWEZKig,5.0,2014-12-27,Excellent food. Excellent service. Excellent a...,0.0,0.0,0.0,"""Cleo""",...,-115.156616,4.5,459.0,1.0,Greek;Middle Eastern;Restaurants;Mediterranean...,Excellent food Excellent service Excellent amb...,"['excellent', 'food', 'excellent', 'service', ...","['excellent', 'food', 'excellent', 'service', ...","[""['excellent',"", ""'food',"", ""'excellent',"", ""...",1
2,w14j-zXZm_JaqAS3VP5XzA,bUsNeErcrACybENrGTPb5g,ee8aiHC6zaY9JoLUVUSc1w,5.0,2011-06-02,Winchell's is always on our list of lunch dest...,1.0,0.0,0.0,"""Winchell's Pub & Grill""",...,-115.103376,4.0,53.0,1.0,Bars;Nightlife;American (Traditional);Restaura...,Winchells is always on our list of lunch desti...,"['winchells', 'is', 'always', 'on', 'our', 'li...","['winchells', 'always', 'list', 'lunch', 'dest...","[""['winchells',"", ""'always',"", ""'list',"", ""'lu...",1
3,-oDFOOAjruW0ZBGsjOVEbQ,46MciGU_hQ3G04CqSBeFxw,u6-m3zvnWn7X162Es8RvPw,2.0,2014-01-14,"For being your neighborhood Ace Store, they do...",0.0,0.0,0.0,"""Southern Highlands Ace Hardware""",...,-115.205466,4.0,30.0,1.0,Home & Garden;Shopping;Hardware Stores,For being your neighborhood Ace Store they don...,"['for', 'being', 'your', 'neighborhood', 'ace'...","['neighborhood', 'ace', 'store', 'dont', 'stoc...","[""['neighborhood',"", ""'ace',"", ""'store',"", ""'d...",0
4,imDHZk__p_yxef_PkA2RIQ,qgbbuhfSkH5_Tna5t9tE-A,FaHADZARwnY4yvlvpnsfGA,1.0,2007-11-10,McCarran has huge revenue available to make an...,1.0,0.0,1.0,"""McCarran International Airport""",...,-115.151009,3.5,3284.0,1.0,Hotels & Travel;Airports,McCarran has huge revenue available to make an...,"['mccarran', 'has', 'huge', 'revenue', 'availa...","['mccarran', 'huge', 'revenue', 'available', '...","[""['mccarran',"", ""'huge',"", ""'revenue',"", ""'av...",0


In [3]:
df_yelp = df_yelp.dropna(axis = 0, how = 'any')

In [4]:
df_yelp['date'] = pd.to_datetime(df_yelp['date'])

In [7]:
df_yelp = df_yelp.sample(frac=1).reset_index(drop=True)

## Test train data split

In [8]:
df_yelptest = df_yelp[800000:] #20% of whole dataset
df_yelptrain = df_yelp[:800000]

In [9]:
X_yelptrain = df_yelptrain['text_nostopwords']
y_yelptrain = df_yelptrain['Positivity']
y_yelptrain2 = df_yelptrain['review_rating']
X_yelptest = df_yelptest['text_nostopwords']
y_yelptest = df_yelptest['Positivity']
y_yelptest2 = df_yelptest['review_rating']

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer().fit(df_yelp['text_nostopwords'])

In [13]:
X_reviewtrain = bow_transformer.transform(df_yelptrain['text_nostopwords'])
X_reviewtest = bow_transformer.transform(df_yelptest['text_nostopwords'])

## Naive Bayes to predict rating

In [14]:
nb_model = MultinomialNB()
nb_model.fit(X_reviewtrain, y_yelptrain2)
Y_pred = nb_model.predict(X_reviewtest)

In [15]:
#print(confusion_matrix(y_test, predict))
print('\n Accuracy for Naive Bayes to predict rating:')
print(accuracy_score(y_yelptest2, Y_pred))
#print(classification_report(y_test, predict))


 Accuracy:
0.589505


## Naive Bayes to predict sentiment

In [16]:
nb_model.fit(X_reviewtrain, y_yelptrain)
Y_pred = nb_model.predict(X_reviewtest)
print('\n Accuracy for Naive Bayes to predict sentiment score:')
print(accuracy_score(y_yelptest, Y_pred))


 Accuracy:
0.848665


## Logistic Regression to predict rating

In [17]:
lg_model = LogisticRegression()
lg_model.fit(X_reviewtrain, y_yelptrain2)
Y_pred_lg = lg_model.predict(X_reviewtest)
#print(accuracy_score(y_yelptest2, Y_pred))

In [18]:
print('\n Accuracy for Logistic Regression to predict rating:')
print(accuracy_score(y_yelptest2, Y_pred_lg))

0.62659

## Logistic Regression to predict sentiment

In [19]:
lg_model.fit(X_reviewtrain, y_yelptrain)
Y_pred_lg = lg_model.predict(X_reviewtest)
print('\n Accuracy for Logistic Regression to predict sentiment score:')
print(accuracy_score(y_yelptest, Y_pred_lg))

0.88522


## Getting categories from the dataset

In [22]:
ref = pd.read_excel("Category_List.xlsx")

In [23]:
ref = ref.groupby(ref.Categories)['SubCategories'].agg(lambda col: ', '.join(col))
ref = pd.DataFrame(ref)
ref=ref.reset_index()

In [24]:
Reference = {}

for i in range(ref.shape[0]):
    Reference[ref.iloc[i,0]] = ref.iloc[i,1].split(", ")

In [25]:
df_yelp['cat_list'] = [i.split(";") for i in df_yelp.categories]

In [26]:
def cat_frequency(cell):
    ## create a dic with all the keys in Reference but values are all 0 
    newDict = {key: 0 for key in Reference.keys()}
    for i in cell:
        for j in Reference:
            if i in Reference[j]:
                ## then if the cat in df shows up in the reference values, the value of new dic plus 1 
                newDict[j] = newDict[j] + 1 
            else:
                continue
    return newDict

In [27]:
df_yelp['cat_freq'] = list(map(cat_frequency, df_yelp.cat_list))

In [28]:
def find_freq_cat(cell):
        return max(cell, key=lambda k: cell[k])

df_yelp['cat_max'] = list(map(find_freq_cat,df_yelp.cat_freq))

In [34]:
df_yelp.to_csv("Yelp_withcategories.csv", index = False)

## Analyzing restaurants

In [31]:
df_res = df_yelp[df_yelp['cat_max'] == 'Restaurants']

In [39]:
df_restest = df_res[int(0.8*df_res.shape[0]):] #20% of whole dataset
df_restrain = df_res[:int(0.8*df_res.shape[0])]

In [42]:
X_restrain = df_restrain['text_nopunct']
y_restrain = df_restrain['Positivity']
y_restrain2 = df_restrain['review_rating']
X_restest = df_restest['text_nopunct']
y_restest = df_restest['Positivity']
y_restest2 = df_restest['review_rating']

In [44]:
bow_transformer_res = CountVectorizer().fit(df_res['text_nopunct'])
X_reviewtrain_res = bow_transformer_res.transform(df_restrain['text_nopunct'])
X_reviewtest_res = bow_transformer_res.transform(df_restest['text_nopunct'])

In [45]:
## Naive Bayes to predict rating
nb_model = MultinomialNB()
nb_model.fit(X_reviewtrain_res, y_restrain2)
Y_pred_res = nb_model.predict(X_reviewtest_res)
print('\n Accuracy for Naive Bayes to predict rating for restaurants:')
print(accuracy_score(y_restest2, Y_pred_res))


0.5926286111953748


In [46]:
## NaiveBayes to predict sentiment score
nb_model.fit(X_reviewtrain_res, y_restrain)
Y_pred_res = nb_model.predict(X_reviewtest_res)
print('\n Accuracy for NaiveBayes to predict sentiment score for restaurants:')
print(accuracy_score(y_restest, Y_pred_res))


 Accuracy:
0.8604057743437841


In [ ]:
## Logistic Regression to predict rating
lg_model = LogisticRegression()
lg_model.fit(X_reviewtrain_res, y_restrain2)
Y_pred_reslg = lg_model.predict(X_reviewtest_res)
print('\n Accuracy for Logistic Regression to predict rating for restaurants:')
print(accuracy_score(y_restest2, Y_pred_reslg))

In [ ]:
## Logistic Regression to predict sentiment score
lg_model.fit(X_reviewtrain_res, y_restrain)
Y_pred_reslg = lg_model.predict(X_reviewtest_res)
print('\n Accuracy for Logistic Regression to predict sentiment score for restaurants:')
print(accuracy_score(y_restest, Y_pred_reslg))

## n-grams

In [5]:
def find_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))

df_yelp['bigrams'] = df_yelp['text_nopunct'].map(lambda x: find_ngrams(x.split(" "), 2))


In [21]:
bigram_vect = CountVectorizer(ngram_range = (1,2)).fit(df_yelp['text_nopunct'])

MemoryError: 

In [11]:
df_ntest = df_yelp[int(0.8*df_yelp.shape[0]):] #20% of whole dataset
df_ntrain = df_yelp[:int(0.8*df_yelp.shape[0])]

In [13]:
X_ntrain = df_ntrain['text_nopunct']
y_ntrain = df_ntrain['Positivity']
y_ntrain2 = df_ntrain['review_rating']
X_ntest = df_ntest['text_nopunct']
y_ntest = df_ntest['Positivity']
y_ntest2 = df_ntest['review_rating']

In [18]:
Xn_ntrain = bigram_vect.transform(df_ntrain['text_nopunct'])
Xn_ntest = bigram_vect.transform(df_ntest['text_nopunct'])

In [19]:
## Naive Bayes to predict rating
nb_model = MultinomialNB()
nb_model.fit(Xn_ntrain, y_ntrain2)
Yn_pred = nb_model.predict(Xn_ntest)
print('\n Accuracy for Naive Bayes to predict rating for n-gram model:')
print(accuracy_score(y_ntest2, Yn_pred))


0.601865


In [22]:
## Naive Bayes to predict sentiment
nb_model = MultinomialNB()
nb_model.fit(Xn_ntrain, y_ntrain)
Yn_pred = nb_model.predict(Xn_ntest)
print('\n Accuracy for Naive Bayes to predict sentiment for n-gram model:')
print(accuracy_score(y_ntest, Yn_pred))


0.87588


In [ ]:
## Logistic Regression to predict rating
lg_model = LogisticRegression()
lg_model.fit(Xn_ntrain, y_ntrain2)
Y_pred_reslg = lg_model.predict(Xn_ntest)
print('\n Accuracy for Logistic Regression to predict rating for n-gram model:')
print(accuracy_score(y_ntest2, Yn_pred))

In [ ]:
## Logistic Regression to predict sentiment
lg_model = LogisticRegression()
lg_model.fit(Xn_ntrain, y_ntrain)
Y_pred_reslg = lg_model.predict(Xn_ntest)
print('\n Accuracy for Logistic Regression to predict sentiment for n-gram model:')
print(accuracy_score(y_ntest, Yn_pred))